In [ ]:
import os
import pandas as pd

In [ ]:
# Read in raw data

# For XRF, ignore repeats and standards
xrf = pd.read_excel('raw/VAS1980.xls',usecols='A:D',index_col=0,header=1)

# For ICPMS, ignnore repeats and standards
icpms = pd.read_excel('raw/vas930.xls',index_col=0,nrows=54)

# Make directory for processed data
os.makedirs('processed',exist_ok=True)

In [ ]:
# Pull unnormalized major elements from XRF and transpose
majors_unnorm = xrf.iloc[4:14,:].T

# Pull normalized major elements from XRF and transpose to make samples index
majors_norm = xrf.iloc[18:28,:].T

# Pull LOI from XRF and transpose 
loi = xrf.iloc[15,:].T

# Remove extra space in majors_unnorm columns
new_cols = majors_unnorm.columns.str.replace(' ','')
new_cols_unnorm = [x+'_unnorm' for x in new_cols]
majors_unnorm_corr = majors_unnorm.copy()
majors_unnorm_corr.columns = new_cols_unnorm

# Remove extra space in majors columns
new_cols = majors_norm.columns.str.replace(' ','')
majors_norm_corr = majors_norm.copy()
majors_norm_corr.columns = new_cols

# Pull XRF trace elements and transpose
xrf_trace = xrf.iloc[31:50,:].T

# Remove extra space in XRF Trace columns
new_cols = xrf_trace.columns.str[1:]
xrf_trace_corr = xrf_trace.copy()
xrf_trace_corr.columns = new_cols

# Remove 'ppm' from ICPMS columns
new_cols = icpms.columns.str[:-4]
icpms_corr = icpms.copy()
icpms_corr.columns  = new_cols

# Remove 'VAS' from ICPMS indices
icpms_indices_stripped = icpms_corr.index.str[4:]
icpms_corr.index = icpms_indices_stripped

# Remove XRF data duplicated by ICPMS
common_cols = xrf_trace_corr.columns.intersection(icpms_corr.columns)
xrf_trace_culled = xrf_trace_corr.drop(common_cols,axis=1)

# Check what is in each file
print(majors_unnorm_corr.columns)
print(majors_norm_corr.columns)
print(loi.name)
print(xrf_trace_culled.index)
print(xrf_trace_culled.columns)
print(xrf_trace_culled.index)
print(icpms_corr.columns)
print(icpms_corr.index)
print(majors_norm_corr.index.equals(xrf_trace_culled.index))
print(xrf_trace_culled.index.equals(icpms_corr.index))


# Combine into single dataframe
data_organized = pd.concat([majors_unnorm_corr,majors_norm_corr,loi,xrf_trace_culled,icpms_corr],axis=1)
print(data_organized.columns)
print(data_organized.index)

In [ ]:
# Read in metadata
meta = pd.read_csv('metadata/gchm_qe.csv',index_col=0)

# Isolate columns of interest
meta_cols = ['Latitude','Longitude','Rock_Type','Period','S_Domain']

# Isolate rows that were actually run
meta_trimmed = meta.loc[data_organized.index,meta_cols]

# Add metadata to main table
data_final = data_organized.join(meta_trimmed)

In [ ]:
# Write to CSV
data_final.to_csv('processed/data_2019.csv')